In [1]:
import os
from dotenv import load_dotenv
from typing import TypedDict, Annotated, Sequence
import operator
from langchain_core.messages import BaseMessage
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.messages import AIMessage, ToolMessage
from langgraph.prebuilt import ToolNode
from langgraph.checkpoint.sqlite import SqliteSaver

# Load API keys and set up tracing
load_dotenv()
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGSMITH_PROJECT"] = "Intro to LangGraph"

# --- Define Tools ---
@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b

@tool
def search_tavily(query: str) -> str:
    """Searches the web for the user's query using the Tavily API."""
    from tavily import TavilyClient
    tavily = TavilyClient(api_key=os.getenv("TAVILY_API_KEY"))
    results = tavily.search(query=query, max_results=2)
    return str(results['results'])

tools = [multiply, search_tavily]
tool_node = ToolNode(tools)

# --- Define State ---
class GraphState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]

# --- Define Nodes and Router ---
def call_llm(state: GraphState):
    llm = ChatOpenAI(model="gpt-4o")
    messages = state['messages']
    llm_with_tools = llm.bind_tools(tools)
    response = llm_with_tools.invoke(messages)
    return {"messages": [response]}

def should_call_tool(state: GraphState):
    last_message = state['messages'][-1]
    if last_message.tool_calls:
        return "call_tool"
    return END

# --- Build the Graph ---
workflow = StateGraph(GraphState)
workflow.add_node("llm", call_llm)
workflow.add_node("call_tool", tool_node)
workflow.set_entry_point("llm")
workflow.add_conditional_edges("llm", should_call_tool)
workflow.add_edge("call_tool", "llm")

# --- Set up Memory ---
# The checkpointer saves the state of the graph after each step
memory = SqliteSaver.from_conn_string(":memory:")

# Compile the graph with the checkpointer to enable memory
app = workflow.compile(checkpointer=memory)

# --- Run the Agent ---
# Use a config with a thread_id to identify the conversation
config = {"configurable": {"thread_id": "my-thread-1"}}

print("--- First turn ---")
for event in app.stream({"messages": [("user", "What's the weather in San Francisco?")]}, config):
    for v in event.values():
        print(v)

print("\n--- Second turn ---")
# Ask a follow-up; the agent should remember the context
for event in app.stream({"messages": [("user", "What about in Paris?")]}, config):
    for v in event.values():
        print(v)

ModuleNotFoundError: No module named 'langgraph.checkpoint.sqlite'